In [3]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm 
import datetime, time, gc 
# from utils import distance, haversine, standard, pad_seq 
from scipy.stats import skew, kurtosis
from zipfile import ZipFile
from collections import Counter 
from sklearn.metrics import roc_auc_score as auc
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score as auc
# import tables

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [13]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in tqdm(df.columns):
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem,
                                                                          100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
# !pip install lightgbm

In [7]:
data = pd.read_pickle('data/train_test_fea.pkl') 
data_tarenc = pd.read_pickle("data/train_test_targetenc.pkl")
print(data.shape, data_tarenc.shape)

data = pd.concat([data, data_tarenc], axis=1)
del data_tarenc
gc.collect()
print(data.shape)

(8000000, 148) (8000000, 14)
(8000000, 162)


In [12]:
cate_cols = ['uid', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
       'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
       'app_first_class', 'app_second_class', 'city', 'age', 
       'device_name', 'career', 'gender', 'net_type',
       'residence', 'his_on_shelf_time', "communication_onlinerate", 
       'up_membership_grade', 'consume_purchase', 'indu_name',
]

for feat in tqdm(cate_cols):
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

use_cols = list( set(data.columns) - set(['label', 'cnt', 'random_sector',
        'uid_task_id_ctr', 'uid_adv_id_ctr', 'pt_d', 'id',
]))

100%|██████████| 24/24 [00:16<00:00,  1.43it/s]


In [15]:
data = reduce_mem_usage(data)

100%|██████████| 162/162 [00:16<00:00,  9.79it/s]

Mem. usage decreased to 2250.24 Mb (35.3% reduction)


In [24]:
# !pip install keras==2.2.4

file_name = datetime.date.today().strftime('%m%d')+"_{}.log".format("deep_base")
def write_log(w):
    t0 = datetime.datetime.now().strftime('%H:%M:%S')
    info = "{} : {}\n".format(t0, w)
    print(info)
    with open(file_name, 'a') as f:
        f.write(info)
        f.write("-"*80+"\n")


test_pred = np.zeros((1000000, ))

# params
params = {
        'learning_rate': 0.05,
        'objective': 'binary',
        'metric': ['binary_logloss', 'auc'], 
        'boosting_type': 'gbdt',
        'num_leaves': 256,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.85,
        'bagging_freq': 3,
        'seed': 8,
        'bagging_seed': 1,
        'feature_fraction_seed': 7,
        'min_data_in_leaf': 20,
        'nthread': 8,
        'verbose': 1,
    }

imp = pd.DataFrame()  # 特征重要性
imp['feat'] = use_cols
    
for i in range(1, 8):
    write_log('pt_d = {}'.format(i))
    
    train = data.iloc[:-1000000][data["pt_d"] != i].reset_index(drop=True)
    valid = data.iloc[:-1000000][data["pt_d"] == i].reset_index(drop=True)
    test = data.iloc[-1000000:].reset_index(drop=True)
    print(train.shape, valid.shape, test.shape)

    # 数据加载
    n_train = lgb.Dataset(train[use_cols], label=train['label'].values, 
                             # free_raw_data=False
                             )
    n_valid = lgb.Dataset(valid[use_cols], label=valid['label'].values, 
                              #free_raw_data=False
                             )

#     del train, valid
    gc.collect()
    
    clf = lgb.train(
            params=params,
            train_set=n_train,
            categorical_feature=cate_cols,
            num_boost_round=5000,
            valid_sets=[n_train, n_valid],
            early_stopping_rounds=30,
            verbose_eval=20,
#             feval=mse_score_eval
        )
    
    train_pred = clf.predict(valid[use_cols], num_iteration=clf.best_iteration)
    
    write_log("val_auc = {}".format(auc(valid['label'].values, train_pred)))
    
    test_pred += clf.predict(test[use_cols], num_iteration=clf.best_iteration) / 7
    
    imp['gain' + str(i)] = clf.feature_importance(importance_type='gain')
    imp['split' + str(i)] = clf.feature_importance(importance_type='split')


11:12:39 : pt_d = 1



/opt/conda/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:39: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/conda/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:40: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


(6000000, 162) (1000000, 162) (1000000, 162)


/opt/conda/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1555: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['adv_id', 'adv_prim_id', 'age', 'app_first_class', 'app_second_class', 'career', 'city', 'communication_onlinerate', 'consume_purchase', 'creat_type_cd', 'dev_id', 'device_name', 'gender', 'his_on_shelf_time', 'indu_name', 'inter_type_cd', 'net_type', 'residence', 'slot_id', 'spread_app_id', 'tags', 'task_id', 'uid', 'up_membership_grade']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Info] Number of positive: 203653, number of negative: 5796347
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.144853 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 31700
[LightGBM] [Info] Number of data points in the train set: 6000000, number of used features: 160


...

(6000000, 162) (1000000, 162) (1000000, 162)


/opt/conda/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1555: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['adv_id', 'adv_prim_id', 'age', 'app_first_class', 'app_second_class', 'career', 'city', 'communication_onlinerate', 'consume_purchase', 'creat_type_cd', 'dev_id', 'device_name', 'gender', 'his_on_shelf_time', 'indu_name', 'inter_type_cd', 'net_type', 'residence', 'slot_id', 'spread_app_id', 'tags', 'task_id', 'uid', 'up_membership_grade']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Info] Number of positive: 206115, number of negative: 5793885
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.155675 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 31984
[LightGBM] [Info] Number of data points in the train set: 6000000, number of used features: 160


/opt/conda/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/opt/conda/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.034353 -> initscore=-3.336124
[LightGBM] [Info] Start training from score -3.336124
Training until validation scores don't improve for 30 rounds
[20]	training's binary_logloss: 0.122151	training's auc: 0.831813	valid_1's binary_logloss: 0.128624	valid_1's auc: 0.797906
[40]	training's binary_logloss: 0.116758	training's auc: 0.847164	valid_1's binary_logloss: 0.126271	valid_1's auc: 0.799847
[60]	training's binary_logloss: 0.113625	training's auc: 0.860233	valid_1's binary_logloss: 0.125943	valid_1's auc: 0.800386
[80]	training's binary_logloss: 0.111073	training's auc: 0.872031	valid_1's binary_logloss: 0.126045	valid_1's auc: 0.800183
Early stopping, best iteration is:
[60]	training's binary_logloss: 0.113625	training's auc: 0.860233	valid_1's binary_logloss: 0.125943	valid_1's auc: 0.800386
12:55:53 : val_auc = 0.8003862523455544



In [19]:
train_pred = clf.predict(valid[use_cols], num_iteration=clf.best_iteration)

# write_log("val_auc = {}".format(auc(valid['label'].values, train_pred)))
    

In [21]:
clf.predict(test[use_cols], num_iteration=clf.best_iteration)

array([0.01792428, 0.08202912, 0.09722099, ..., 0.03920117, 0.01852832,
       0.02759261])

In [26]:
result = pd.DataFrame({
        'id': np.load("data/test_id.npy"), 
        'probability': test_pred.reshape(-1), 
    })

result.to_csv("submission.csv", index=False) 